In [89]:
# Import dependencies

import gmaps
import pandas as pd
import numpy as np
import requests
import json

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# Import OpenWeather API key
from config import api_key

In [42]:
# Read in weather csv
weather_df = pd.read_csv('../WeatherPy/weather.csv')
# # Drop na values
# weather_df.dropna(inplace = True) 
# Drop unnamed column
weather_df.drop(["Unnamed: 0"], axis=1,inplace=True)
weather_df.tail()

,city,latitude,longitude,temperature,humidity,wind speed,cloudiness
457,saint-philippe,-21.36,55.77,22.45,54.0,7.20,75
458,hobart,-42.88,147.33,12.84,82.0,1.86,75
459,zhaotong,27.32,103.72,16.75,52.0,2.97,0
460,sorland,67.67,12.69,7.83,91.0,11.21,100
461,bluff,-46.60,168.33,22.78,59.0,1.34,100


In [43]:
# Store latitude and longitude in locations
locations = weather_df[["latitude","longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["humidity"].astype(float)

In [45]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [62]:
# Narrow down weather df based on ideal weather conditions:
# Temp < 30; > 22
# Wind speed < 5
# Cloudiness < 15

ideal_weather_df = weather_df.loc[
    (weather_df["temperature"]>22)
    & (weather_df["temperature"]<30)
    & (weather_df["wind speed"]<5)
    & (weather_df["cloudiness"]<15)
]

ideal_weather_df


,city,latitude,longitude,temperature,humidity,wind speed,cloudiness
7,coahuayana,18.73,-103.68,23.17,86.0,1.18,1
28,inhambane,-23.86,35.38,24.53,62.0,3.68,1
50,batemans bay,-35.72,150.18,26.36,38.0,0.89,0
63,flinders,-34.58,150.86,25.33,65.0,2.24,0
130,immokalee,26.42,-81.42,23.04,94.0,1.50,1
142,vila velha,-20.33,-40.29,24.70,88.0,0.50,0
162,makaha,21.47,-158.22,25.53,73.0,2.60,1
234,zhangye,38.93,100.45,22.04,17.0,4.06,1
279,makakilo city,21.35,-158.09,25.93,61.0,2.10,1
320,vilhena,-12.74,-60.15,22.17,82.0,1.56,3


In [64]:
# Set up additional columns to hold information
ideal_weather_df['Hotel Name'] = ""
ideal_weather_df['City'] = ""
ideal_weather_df['Country'] = ""

ideal_weather_df.head()

,city,latitude,longitude,temperature,humidity,wind speed,cloudiness,Hotel Name,City,Country
7,coahuayana,18.73,-103.68,23.17,86.0,1.18,1,,,
28,inhambane,-23.86,35.38,24.53,62.0,3.68,1,,,
50,batemans bay,-35.72,150.18,26.36,38.0,0.89,0,,,
63,flinders,-34.58,150.86,25.33,65.0,2.24,0,,,
130,immokalee,26.42,-81.42,23.04,94.0,1.50,1,,,


In [81]:
# Test retrival of hotel from Google Places API

# Find the closest hotel to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-31.94889169999999,115.8622556",  # The Old Swan Barracks Backpacker Hostel Perth,
    "rankby": "distance",
    "type": "lodging",
    "key": gkey,
}

response = requests.get(base_url, params=params).json()
# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

results = response['results']

print(f"Closest hotel is {results[0]['name']}.")


Closest hotel is The Old Swan Barracks Backpacker Hostel Perth.


In [84]:
# Retrieve all hotels

# Find the closest hotel to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-31.94889169999999,115.8622556",  # The Old Swan Barracks Backpacker Hostel Perth,
    "rankby": "distance",
    "type": "lodging",
    "key": gkey,
}

# Use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_df.iterrows():

    # Get lat and long from df
    latitude = row['latitude']
    longitude = row['longitude']

    # Add keyword to params dict
    params['location'] = f"{latitude},{longitude}"

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {latitude},{longitude}.")
    response = requests.get(base_url, params=params).json()

    # Extract results
    results = response['results']

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        ideal_weather_df.loc[index, 'Hotel Name'] = results[0]['name']
        ideal_weather_df.loc[index, 'City'] = results[0]['vicinity']
        # ideal_weather_df.loc[index, 'Country'] = results[0]['price_level']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 7: 18.73,-103.68.
Closest hotel is Hotel Los Arcos.
------------
Retrieving Results for Index 28: -23.86,35.38.
Closest hotel is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 50: -35.72,150.18.
Closest hotel is Shady Willows Caravan Park.
------------
Retrieving Results for Index 63: -34.58,150.86.
Closest hotel is Shellharbour Village Escape.
------------
Retrieving Results for Index 130: 26.42,-81.42.
Closest hotel is Budget Inn.
------------
Retrieving Results for Index 142: -20.33,-40.29.
Closest hotel is Hotel Prainha.
------------
Retrieving Results for Index 162: 21.47,-158.22.
Closest hotel is The Bungalows Hawaii.
------------
Retrieving Results for Index 234: 38.93,100.45.
Closest hotel is 县府街名烟名酒店.
------------
Retrieving Results for Index 279: 21.35,-158.09.
Closest hotel is エレン.
------------
Retrieving Results for Index 320: -12.74,-60.15.
Closest hotel is Hotel Pirâmide.
------------
Retrieving Results for Index 336: -3.84,-

In [87]:
# Add column to df for country
ideal_weather_df["Country"] = ""

In [90]:
# Use Open Weather Map API to retrieve country

base_url = "http://api.openweathermap.org/data/2.5/weather"
params = {
    "q": "albany",
    "appid": api_key,
    "units": "metric"
}

# Use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_df.iterrows():

    # Get city from weather_df
    city = row['city']

    # Add query to params dict
    params['q'] = city

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}")

    # Extract results
    response = requests.get(base_url, params=params).json()
    
    try:
        ideal_weather_df.loc[index, "Country"] = response["sys"]["country"]
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 7: coahuayana
------------
Retrieving Results for Index 28: inhambane
------------
Retrieving Results for Index 50: batemans bay
------------
Retrieving Results for Index 63: flinders
------------
Retrieving Results for Index 130: immokalee
------------
Retrieving Results for Index 142: vila velha
------------
Retrieving Results for Index 162: makaha
------------
Retrieving Results for Index 234: zhangye
------------
Retrieving Results for Index 279: makakilo city
------------
Retrieving Results for Index 320: vilhena
------------
Retrieving Results for Index 336: codajas
------------
Retrieving Results for Index 391: savelugu
------------
Retrieving Results for Index 444: salalah
------------


In [91]:
ideal_weather_df.head()


,city,latitude,longitude,temperature,humidity,wind speed,cloudiness,Hotel Name,City,Country
7,coahuayana,18.73,-103.68,23.17,86.0,1.18,1,Hotel Los Arcos,"Centro, Coahuayana de Hidalgo",MX
28,inhambane,-23.86,35.38,24.53,62.0,3.68,1,Hotel Casa Do Capitão,"Balane 1, Av. Maguiguana, Inhambane",MZ
50,batemans bay,-35.72,150.18,26.36,38.0,0.89,0,Shady Willows Caravan Park,"South Street, Batemans Bay",AU
63,flinders,-34.58,150.86,25.33,65.0,2.24,0,Shellharbour Village Escape,"9 Robertson Street, Shellharbour",AU
130,immokalee,26.42,-81.42,23.04,94.0,1.50,1,Budget Inn,"504 East Main Street, Immokalee",US
